In [2]:
import glob
filenames=glob.glob("data/train_data/Conll_Format/*.txt")
with open('train.txt', 'w',encoding="utf-8") as outfile:
    for fname in filenames:
        with open(fname,'r+',encoding="utf-8") as infile:
            outfile.write(infile.read())

In [3]:
 import glob
filenames=glob.glob("data/dev_data/Conll_Format/*.txt")
with open('validation.txt', 'w',encoding="utf-8") as outfile:
    for fname in filenames:
        with open(fname,'r+',encoding="utf-8") as infile:
            outfile.write(infile.read())

In [4]:
import glob
filenames=glob.glob("data/test_data/Conll_Format/*.txt")
with open('test.txt', 'w',encoding="utf-8") as outfile:
    for fname in filenames:
        with open(fname,'r+',encoding="utf-8") as infile:
            outfile.write(infile.read())

In [5]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token

            ######################################
            ######### YOUR CODE HERE #############
            ######################################
            if(token[0:8]=="https://" or token[0:7]=="http://"):
                token = "<URL>"
            elif(token[0]=="@"):
                token = "<USR>"
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [6]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [7]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

Illumina	B-Modifier
Small	I-Modifier
RNA	B-Reagent
cloning	B-Action
protocol	O
using	O
Random	B-Modifier
Adapters	B-Reagent

Section	O
I	O
:	O
Size	B-Method
selection	I-Method
and	O
Gel	B-Reagent
purification	B-Method
of	O
RNA	B-Reagent
sample	I-Reagent
.	O

Prepare	B-Action
a	O
12	B-Concentration
%	I-Concentration
polyacrylamide/urea	B-Reagent
gel	I-Reagent



### Prepare dictionaries

To train a neural network, we will use two mappings: 
- {token}$\to${token id}: address the row in embeddings matrix for the current token;
- {tag}$\to${tag id}: one-hot ground truth probability distribution vectors for computing the loss at the output of the network.

Now you need to implement the function *build_dict* which will return {token or tag}$\to${index} and vice versa. 

In [8]:
from collections import defaultdict

In [9]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
   
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    for token in special_tokens:
        idx2tok.append(token)
        
    for tokens in tokens_or_tags:
        for token in tokens:
            idx2tok.append(token)
    idx2tok = list(set(idx2tok))
    
    for i in range(len(idx2tok)):
        tok2idx[idx2tok[i]]=i 
    
            
    
    return tok2idx, idx2tok

After implementing the function *build_dict* you can make dictionaries for tokens and tags. Special tokens in our case will be:
 - `<UNK>` token for out of vocabulary tokens;
 - `<PAD>` token for padding sentence to the same length when we create batches of sentences.

In [10]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [11]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

### Generate batches

Neural Networks are usually trained with batches. It means that weight updates of the network are based on several sequences at every single time. The tricky part is that all sequences within a batch need to have the same length. So we will pad them with a special `<PAD>` token. It is also a good practice to provide RNN with sequence lengths, so it can skip computations for padding parts. We provide the batching function *batches_generator* readily available for you to save time. 

In [12]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

In [13]:
import tensorflow as tf
import numpy as np

In [14]:
class BiLSTMModel():
    pass

First, we need to create [placeholders](https://www.tensorflow.org/versions/master/api_docs/python/tf/placeholder) to specify what data we are going to feed into the network during the execution time.  For this task we will need the following placeholders:
 - *input_batch* — sequences of words (the shape equals to [batch_size, sequence_len]);
 - *ground_truth_tags* — sequences of tags (the shape equals to [batch_size, sequence_len]);
 - *lengths* — lengths of not padded sequences (the shape equals to [batch_size]);
 - *dropout_ph* — dropout keep probability; this placeholder has a predefined value 1;
 - *learning_rate_ph* — learning rate; we need this placeholder because we want to change the value during training.

It could be noticed that we use *None* in the shapes in the declaration, which means that data of any size can be feeded. 

You need to complete the function *declare_placeholders*.

In [15]:
def declare_placeholders(self):
    """Specifies placeholders for the model."""

    # Placeholders for input and ground truth output.
    self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
    self.ground_truth_tags = tf.placeholder(dtype = tf.int32, shape=[None, None], name='ground_truth_tags')
  
    # Placeholder for lengths of the sequences.
    self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 
    
    # Placeholder for a dropout keep probability. If we don't feed
    # a value for this placeholder, it will be equal to 1.0.
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    
    # Placeholder for a learning rate (tf.float32).
    self.learning_rate_ph = tf.placeholder(dtype='float32', shape=[], name='learning_rate_ph')

In [16]:
BiLSTMModel.__declare_placeholders = classmethod(declare_placeholders)

In [17]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    """Specifies bi-LSTM architecture and computes logits for inputs."""
    
    # Create embedding variable (tf.Variable) with dtype tf.float32
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.Variable(initial_embedding_matrix,name='embedding_matrix',dtype=tf.float32)
    
    # Create RNN cells (for example, tf.nn.rnn_cell.BasicLSTMCell) with n_hidden_rnn number of units 
    # and dropout (tf.nn.rnn_cell.DropoutWrapper), initializing all *_keep_prob with dropout placeholder.
    forward_cell =  tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.BasicLSTMCell(n_hidden_rnn),
                                                  input_keep_prob=self.dropout_ph,
                                                  output_keep_prob=self.dropout_ph,
                                                  state_keep_prob=self.dropout_ph)
    backward_cell =  tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.BasicLSTMCell(n_hidden_rnn),
                                                  input_keep_prob=self.dropout_ph,
                                                  output_keep_prob=self.dropout_ph,
                                                  state_keep_prob=self.dropout_ph)

    # Look up embeddings for self.input_batch (tf.nn.embedding_lookup).
    # Shape: [batch_size, sequence_len, embedding_dim].
    embeddings =  tf.nn.embedding_lookup(embedding_matrix_variable,self.input_batch)
    
    # Pass them through Bidirectional Dynamic RNN (tf.nn.bidirectional_dynamic_rnn).
    # Shape: [batch_size, sequence_len, 2 * n_hidden_rnn]. 
    # Also don't forget to initialize sequence_length as self.lengths and dtype as tf.float32.
    (rnn_output_fw, rnn_output_bw), _ =  tf.nn.bidirectional_dynamic_rnn(forward_cell,backward_cell,embeddings,sequence_length=self.lengths,dtype=tf.float32)
    rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

    # Dense layer on top.
    # Shape: [batch_size, sequence_len, n_tags].   
    self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)

In [18]:
BiLSTMModel.__build_layers = classmethod(build_layers)

In [19]:
def compute_predictions(self):
    """Transforms logits to probabilities and finds the most probable tags."""
    
    # Create softmax (tf.nn.softmax) function
    softmax_output = tf.nn.softmax(self.logits)
    
    # Use argmax (tf.argmax) to get the most probable tags
    # Don't forget to set axis=-1
    # otherwise argmax will be calculated in a wrong way
    self.predictions = tf.argmax(softmax_output,axis=-1)

In [20]:
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

In [21]:
def compute_loss(self, n_tags, PAD_index):
    """Computes masked cross-entopy loss with logits."""
    
    # Create cross entropy function function (tf.nn.softmax_cross_entropy_with_logits)
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor =  tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=ground_truth_tags_one_hot)
    
    mask = tf.cast(tf.not_equal(self.input_batch, PAD_index), tf.float32)
    # Create loss function which doesn't operate with <PAD> tokens (tf.reduce_mean)
    # Be careful that the argument of tf.reduce_mean should be
    # multiplication of mask and loss_tensor.
    self.loss =  tf.reduce_mean(tf.multiply(mask,loss_tensor))

In [22]:
BiLSTMModel.__compute_loss = classmethod(compute_loss)

In [23]:
def perform_optimization(self):
    """Specifies the optimizer and train_op for the model."""
    
    # Create an optimizer (tf.train.AdamOptimizer)
    self.optimizer =  tf.train.AdamOptimizer(learning_rate=self.learning_rate_ph)
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    
    # Gradient clipping (tf.clip_by_norm) for self.grads_and_vars
    # Pay attention that you need to apply this operation only for gradients 
    # because self.grads_and_vars contains also variables.
    # list comprehension might be useful in this case.
    clip_norm = tf.cast(1.0, tf.float32)
    self.grads_and_vars =  [(tf.clip_by_norm(i,clip_norm),j) for i,j in self.grads_and_vars] 
    
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

In [24]:
BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [25]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

In [26]:
BiLSTMModel.__init__ = classmethod(init_model)

## Train the network and predict tags

In [27]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {self.input_batch: x_batch,
                 self.ground_truth_tags: y_batch,
                 self.learning_rate_ph: learning_rate,
                 self.dropout_ph: dropout_keep_probability,
                 self.lengths: lengths}
    
    session.run(self.train_op, feed_dict=feed_dict)

In [28]:
BiLSTMModel.train_on_batch = classmethod(train_on_batch)

Implement the function *predict_for_batch* by initializing *feed_dict* with input *x_batch* and *lengths* and running the *session* for *self.predictions*.

In [29]:
def predict_for_batch(self, session, x_batch, lengths):
    predictions = session.run(self.predictions,feed_dict={self.input_batch:x_batch,self.lengths:lengths})
    
    return predictions

In [30]:
BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

In [31]:
from evaluation import precision_recall_f1

In [32]:
def predict_tags(model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

## Run your experiment

Create *BiLSTMModel* model with the following parameters:
 - *vocabulary_size* — number of tokens;
 - *n_tags* — number of tags;
 - *embedding_dim* — dimension of embeddings, recommended value: 200;
 - *n_hidden_rnn* — size of hidden layers for RNN, recommended value: 200;
 - *PAD_index* — an index of the padding token (`<PAD>`).

Set hyperparameters. You might want to start with the following recommended values:
- *batch_size*: 32;
- 4 epochs;
- starting value of *learning_rate*: 0.005
- *learning_rate_decay*: a square root of 2;
- *dropout_keep_probability*: try several values: 0.1, 0.5, 0.9.

However, feel free to conduct more experiments to tune hyperparameters and earn extra points for the assignment.

In [33]:
tf.reset_default_graph()

model = BiLSTMModel(len(token2idx),len(tag2idx),embedding_dim=200,n_hidden_rnn=200,PAD_index=token2idx["<PAD>"])

batch_size = 32
n_epochs = 4
learning_rate = 0.005
learning_rate_decay = np.sqrt(2)
dropout_keep_probability = 0.9

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



If you got an error *"Tensor conversion requested dtype float64 for Tensor with dtype float32"* in this point, check if there are variables without dtype initialised. Set the value of dtype equals to *tf.float32* for such variables.

Finally, we are ready to run the training!

In [34]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    # For each epoch evaluate the model on train and validation data
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size, train_tokens, train_tags):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

-------------------- Epoch 1 of 4 --------------------
Train data evaluation:
processed 132515 tokens with 47604 phrases; found: 97095 phrases; correct: 1390.

precision:  1.43%; recall:  2.92%; F1:  1.92

Validation data evaluation:
processed 44604 tokens with 15846 phrases; found: 32712 phrases; correct: 440.

precision:  1.35%; recall:  2.78%; F1:  1.81

-------------------- Epoch 2 of 4 --------------------
Train data evaluation:
processed 132515 tokens with 47604 phrases; found: 49824 phrases; correct: 37418.

precision:  75.10%; recall:  78.60%; F1:  76.81

Validation data evaluation:
processed 44604 tokens with 15846 phrases; found: 16919 phrases; correct: 11181.

precision:  66.09%; recall:  70.56%; F1:  68.25

-------------------- Epoch 3 of 4 --------------------
Train data evaluation:
processed 132515 tokens with 47604 phrases; found: 49160 phrases; correct: 39625.

precision:  80.60%; recall:  83.24%; F1:  81.90

Validation data evaluation:
processed 446

Now let us see full quality reports for the final model on train, validation, and test sets. To give you a hint whether you have implemented everything correctly, you might expect F-score about 40% on the validation set.

**The output of the cell below (as well as the output of all the other cells) should be present in the notebook for peer2peer review!**

In [35]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_tokens, train_tags, short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess,validation_tokens, validation_tags, short_report=False)

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_tokens, test_tags, short_report=False)

-------------------- Train set quality: --------------------
processed 132515 tokens with 47604 phrases; found: 48508 phrases; correct: 42546.

precision:  87.71%; recall:  89.37%; F1:  88.53

	      Action: precision:   94.63%; recall:   95.39%; F1:   95.01; predicted:  12455

	      Amount: precision:   89.19%; recall:   91.63%; F1:   90.40; predicted:  3498

	Concentration: precision:   86.22%; recall:   89.20%; F1:   87.68; predicted:  1379

	      Device: precision:   86.31%; recall:   86.61%; F1:   86.46; predicted:  1753

	Generic-Measure: precision:   67.26%; recall:   69.61%; F1:   68.42; predicted:   504

	    Location: precision:   83.32%; recall:   88.71%; F1:   85.93; predicted:  4178

	Measure-Type: precision:   71.11%; recall:   75.44%; F1:   73.21; predicted:   907

	     Mention: precision:   88.26%; recall:   73.15%; F1:   80.00; predicted:   213

	      Method: precision:   72.57%; recall:   76.06%; F1:   74.28; predicted:  1677

	    Modifier: precision:   77.70%; r

In [48]:
a,b=read_data('data/test.txt')

In [62]:
for x_batch, y_batch, lengths in batches_generator(1, a, b):
        tags_batch, tokens_batch = predict_tags(model, sess, x_batch, lengths)
        print(tokens_batch)
        print(tags_batch)

[['Apply', '100', 'µl', 'an', 'appropriately', 'diluted', 'primary', 'antibody', '(', 'in', 'antibody', 'dilution', 'buffer', ',', 'e.g', '.', '0.5', '%', 'bovine', 'serum', 'albumin', 'in', 'PBS', ')', 'to', 'the', 'sections', 'on', 'the', 'slides', 'and', 'incubate', 'in', 'a', 'humidified', 'chamber', 'for', '1', 'h', 'at', 'room', 'temperature', 'or', 'overnight', 'at', '4°C', '.']]
[['B-Action', 'B-Amount', 'I-Amount', 'O', 'B-Modifier', 'I-Modifier', 'B-Reagent', 'I-Reagent', 'O', 'O', 'B-Reagent', 'I-Reagent', 'I-Reagent', 'O', 'O', 'O', 'B-Concentration', 'I-Concentration', 'B-Reagent', 'I-Reagent', 'I-Reagent', 'O', 'B-Reagent', 'O', 'O', 'O', 'B-Reagent', 'O', 'O', 'B-Location', 'O', 'B-Action', 'O', 'O', 'B-Device', 'I-Device', 'O', 'B-Time', 'I-Time', 'O', 'B-Temperature', 'I-Temperature', 'O', 'B-Time', 'O', 'B-Temperature', 'O']]
[['Mix', 'well', 'and', 'hold', 'at', '-20°C', 'overnight', '.']]
[['B-Action', 'B-Modifier', 'O', 'B-Action', 'O', 'B-Temperature', 'B-Time', '

[['Once', 'enriched', 'sufficiently', ',', 'filter', 'out', 'cells', 'with', '0.2um', 'filtration', 'into', '1.5ml', 'microfuge', 'tubes', 'Spin', 'for', '5', 'mins', '.', 'at', 'max', 'to', 'pellet', 'cells', '.']]
[['B-Modifier', 'I-Modifier', 'I-Modifier', 'O', 'B-Action', 'I-Action', 'B-Reagent', 'O', 'B-Size', 'B-Device', 'O', 'B-Amount', 'B-Location', 'I-Location', 'B-Action', 'O', 'B-Time', 'I-Time', 'O', 'O', 'B-Speed', 'O', 'B-Action', 'B-Reagent', 'O']]
[['Mix:3.7', 'g', 'NH4F10', 'ml', 'H2O']]
[['B-Action', 'B-Amount', 'B-Reagent', 'B-Amount', 'B-Reagent']]
[['Filter', 'sterilize', 'each', 'solution', 'using', '0.2', 'µm', 'syringe', 'filter', 'into', 'new', ',', 'sterile', '15', 'or', '50', 'ml', 'centrifuge', 'tubes', 'or', 'acid', 'washed', 'and', 'sterile', 'polycarbonate', 'bottles', '.']]
[['B-Action', 'I-Action', 'B-Modifier', 'B-Reagent', 'O', 'B-Size', 'I-Size', 'B-Device', 'I-Device', 'O', 'B-Modifier', 'O', 'B-Modifier', 'B-Amount', 'O', 'B-Amount', 'I-Amount', 'B

[['Wash', '#', '2', ':', 'Add', '180', 'µL', 'of', 'freshly', 'prepared', '80', '%', 'ethanol', ',', 'incubate', 'for', '30', 'seconds', ',', 'and', 'remove', 'with', 'a', 'P200', 'or', 'likely', 'set', 'to', '200', 'µL', '.']]
[['B-Action', 'O', 'O', 'O', 'B-Action', 'B-Amount', 'I-Amount', 'I-Amount', 'B-Modifier', 'I-Modifier', 'B-Concentration', 'I-Concentration', 'B-Reagent', 'B-Reagent', 'B-Action', 'O', 'B-Time', 'I-Time', 'O', 'O', 'B-Action', 'O', 'O', 'B-Device', 'O', 'O', 'O', 'O', 'B-Amount', 'I-Amount', 'I-Amount']]
[['Incubate', 'at', '37°C', 'for', '30', 'minutes', 'with', 'periodic', 'mixing', '.']]
[['B-Action', 'O', 'B-Temperature', 'O', 'B-Time', 'I-Time', 'O', 'B-Method', 'I-Method', 'O']]
[['Carefully', 'remove', 'and', 'discard', 'the', 'supernatant', '.']]
[['B-Modifier', 'B-Action', 'O', 'B-Action', 'O', 'B-Reagent', 'O']]
[['Submerge', 'protein', 'acrylamide', 'gel', 'in', 'coomassie', 'blue', 'stain', 'and', 'shake', 'at', 'room', 'temperature', 'for', 'two', 

[['To', 'carry', 'out', 'the', 'taxonomic', 'assignment', ',', 'these', 'non-assembled', 'read', 'data', 'sets', 'were', 'uploaded', 'to', 'the', 'Metavir', 'web', 'server', ',', 'which', 'is', 'dedicated', 'to', 'the', 'analysis', 'of', 'viral', 'metagenomes', '(', 'http', ':', '//metavir-meb.univ-bpclermont.fr', ')', '(', 'Roux', 'et', 'al.', ',', '2011', ')', '.']]
[['O', 'B-Action', 'I-Action', 'O', 'B-Method', 'I-Method', 'O', 'O', 'B-Modifier', 'B-Reagent', 'I-Reagent', 'I-Reagent', 'O', 'B-Action', 'O', 'O', 'B-Method', 'I-Method', 'I-Method', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Method', 'O', 'B-Reagent', 'I-Reagent', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['Place', 'the', 'tube', 'in', 'the', 'magnet', 'for', '5', 'minutes', '.']]
[['B-Action', 'O', 'B-Location', 'O', 'O', 'B-Device', 'O', 'B-Time', 'I-Time', 'O']]
[['Centrifuge', 'the', 'DNA', 'samples', 'in', 'the', 'Sorvall', 'HB-4', 'rotor', 'at', '10,000', 'rpm', ',', '20', 'min', ',', '4°C', '.']]

[['Plaque', 'Assay', 'Protocol', 'for', 'Pseudoalteromonas']]
[['B-Method', 'I-Method', 'I-Method', 'O', 'B-Reagent']]
[['Add', '1', 'ml', 'of', 'likely', 'Buffer', '(', '0.2', 'M', 'likely', ',', '[', 'pH', '8.2', ']', ')', 'to', 'the', 'Protein', 'A/G', 'immobilized', 'antibody', '.']]
[['B-Action', 'B-Amount', 'I-Amount', 'O', 'B-Reagent', 'I-Reagent', 'O', 'B-Generic-Measure', 'I-Generic-Measure', 'O', 'O', 'O', 'B-pH', 'I-pH', 'O', 'O', 'O', 'O', 'B-Reagent', 'I-Location', 'B-Modifier', 'B-Reagent', 'O']]
[['Centrifuge', 'the', 'gradient', 'at', '20,000x', 'g', 'for', '20', 'minutes', '.']]
[['B-Action', 'O', 'B-Reagent', 'O', 'B-Speed', 'I-Speed', 'O', 'B-Time', 'I-Time', 'O']]
[['Centrifuge', '10,000', 'g', 'for', '2min', 'to', 'remove', 'any', 'residual', 'liquid', '.']]
[['B-Action', 'B-Speed', 'I-Speed', 'O', 'B-Time', 'O', 'B-Action', 'B-Modifier', 'B-Modifier', 'B-Reagent', 'O']]
[['Rinse', 'the', 'slides', 'in', '300', 'ml', 'of', '10', 'mM', 'phosphate', 'buffered', 'sali

...

[['[', '6.4', ']', 'Heat', 'the', 'likely', 'to', '95', '°C', 'for', '90', 'sec', '(', 'e.g.', ',', 'using', 'a', 'PCR', 'likely', 'cycler', ')', '.']]
[['O', 'O', 'O', 'B-Action', 'O', 'O', 'O', 'B-Temperature', 'I-Temperature', 'O', 'B-Time', 'I-Time', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Device', 'O', 'O']]
[['Change', 'by', 'rinsing', 'slides', 'in', '300', 'ml', 'PBS', 'for', '5', 'min', '.', '(', '1/2', ')']]
[['B-Action', 'O', 'B-Action', 'B-Location', 'O', 'B-Amount', 'I-Amount', 'B-Reagent', 'O', 'B-Time', 'I-Time', 'O', 'O', 'O', 'O']]
[['Transfer', '1', 'dish', 'of', 'day4', 'Ebs', 'into', '15ml', 'tube', '.']]
[['B-Action', 'B-Amount', 'I-Amount', 'O', 'B-Modifier', 'B-Reagent', 'O', 'B-Amount', 'B-Location', 'O']]
[['Set', 'pipette', 'to', '88', 'μL', 'and', 'gently', 'remove', 'clear', 'supernatant', 'taking', 'care', 'not', 'to', 'disturbbeads', '.']]
[['B-Action', 'I-Action', 'O', 'B-Amount', 'I-Amount', 'O', 'B-Modifier', 'B-Action', 'B-Modifier', 'B-Reagent', 'O', 'O

[['Transfer', 'the', 'frozen', 'tissue', 'block', 'to', 'a', 'likely', 'cryostat', '(', 'e.g', '.', '-20°C', ')', 'prior', 'to', 'sectioning', 'and', 'allow', 'the', 'temperature', 'of', 'the', 'frozen', 'tissue', 'block', 'to', 'equilibrate', 'to', 'the', 'temperature', 'of', 'the', 'likely', 'cryostat', '.']]
[['B-Action', 'O', 'B-Modifier', 'B-Reagent', 'I-Reagent', 'O', 'O', 'O', 'B-Device', 'O', 'O', 'O', 'B-Temperature', 'O', 'O', 'O', 'B-Action', 'O', 'B-Action', 'O', 'B-Measure-Type', 'O', 'O', 'B-Modifier', 'B-Reagent', 'I-Reagent', 'O', 'B-Action', 'O', 'O', 'B-Measure-Type', 'O', 'O', 'O', 'B-Device', 'O']]
[['Wash', '2X', 'with', 'BioLegend', '’', 's', 'Cell', 'Staining', 'Buffer', '(', 'Cat', '.', 'No', '.', '420201', ')', 'and', 'then', 'resuspend', 'in', '0.5', 'ml', 'cell', 'staining', 'buffer', 'for', 'flow', 'likely', 'analysis', '.']]
[['B-Action', 'B-Numerical', 'O', 'B-Reagent', 'I-Reagent', 'I-Reagent', 'I-Reagent', 'I-Reagent', 'I-Reagent', 'O', 'O', 'O', 'O', 'O

[['After', 'determining', 'the', 'concentration', 'of', 'the', 'FLVs', 'in', 'the', 'concentrate', ',', 'and', 'the', 'ambient', 'concentration', 'of', 'viruses', 'in', 'the', 'seawater', ',', 'the', 'proper', 'amount', 'of', 'FLV', 'concentrate', 'should', 'be', 'added', 'at', 'tracer', 'levels', '(', '<', '10', '%', 'of', 'original', 'ambient', 'virus', 'concentration', ')', 'into', 'sample', 'volumes', 'of', 'no', 'less', 'than', '400', 'mL', '.']]
[['O', 'B-Action', 'O', 'B-Measure-Type', 'O', 'O', 'O', 'O', 'O', 'B-Reagent', 'O', 'O', 'O', 'B-Reagent', 'I-Reagent', 'O', 'B-Reagent', 'O', 'O', 'B-Reagent', 'O', 'O', 'O', 'O', 'O', 'B-Reagent', 'I-Reagent', 'O', 'O', 'B-Action', 'O', 'B-Generic-Measure', 'O', 'O', 'B-Modifier', 'B-Concentration', 'I-Concentration', 'O', 'O', 'B-Reagent', 'I-Reagent', 'B-Measure-Type', 'O', 'O', 'B-Reagent', 'B-Measure-Type', 'O', 'O', 'O', 'I-Modifier', 'B-Amount', 'I-Amount', 'O']]
[['From', 'each', 'of', 'these', 'likely', 'bottles', ',', 'triplic

[['Keep', 'sample', 'tube', 'on', 'the', 'magnetic', 'stand', '.']]
[['B-Action', 'B-Location', 'I-Location', 'O', 'O', 'B-Location', 'I-Location', 'O']]
[['Place', 'the', 'cell', 'suspension', 'in', 'an', 'ice', 'bath', 'for', '15', 'min', '.']]
[['B-Action', 'O', 'B-Reagent', 'I-Reagent', 'O', 'O', 'B-Location', 'I-Location', 'O', 'B-Time', 'I-Time', 'O']]
[['Remove', 'supernatant', '.']]
[['B-Action', 'B-Reagent', 'O']]
[['Add', '10', 'μL', 'of', 'Streptavidin', 'Nanobeads', '.']]
[['B-Action', 'B-Amount', 'I-Amount', 'O', 'B-Reagent', 'I-Reagent', 'O']]
[['Add', '2.5', 'U', 'Klenow', 'fragment', 'and', '5', 'U', 'T4', 'DNA', 'polymerase', '.']]
[['B-Action', 'B-Amount', 'I-Amount', 'B-Reagent', 'I-Reagent', 'O', 'B-Amount', 'I-Amount', 'B-Reagent', 'I-Reagent', 'I-Reagent', 'O']]
[['Centrifuge', 'to', 'pellet', 'cells', 'and', 'aspirate', 'solution', '.']]
[['B-Action', 'O', 'B-Action', 'B-Reagent', 'O', 'B-Action', 'B-Reagent', 'O']]
[['Collect', 'seawater', 'samples', 'from', 'th

[['Vortex', 'or', 'pipet', 'to', 'ensure', 'cells', 'are', 'completely', 'resuspended', '.']]
[['B-Action', 'O', 'B-Action', 'O', 'B-Action', 'B-Reagent', 'O', 'B-Modifier', 'I-Modifier', 'O']]
[['Thaw', 'likely', 'mg', 'of', 'tissue', 'or', 'likely', 'dish', 'cells', 'in', '1', 'mL', 'of', 'high', 'salt', 'buffer', '.']]
[['B-Action', 'B-Amount', 'I-Amount', 'O', 'B-Reagent', 'O', 'O', 'B-Location', 'B-Reagent', 'O', 'B-Amount', 'I-Amount', 'O', 'B-Reagent', 'I-Reagent', 'I-Reagent', 'O']]
[['Centrifuge', 'at', 'likely', 'for', 'likely', 'hr', 'at', '18°C', '.']]
[['B-Action', 'O', 'O', 'O', 'B-Time', 'I-Time', 'O', 'B-Temperature', 'O']]
[['Set', 'up', 'the', 'following', 'cDNA', 'Synthesis', 'reaction', 'on', 'ice', ':', '3', '’', 'and', '5', '’', 'Adapter', 'Ligated', 'RNA', 'from', 'Step', '16', '(', 'contains', 'TPB', ')', ',', 'likely', ';', 'Mix', 'likely', ',', '2μL', ';', 'Mix', 'likely', '1μL', ';']]
[['B-Action', 'I-Action', 'O', 'O', 'B-Reagent', 'B-Method', 'I-Method', 'O

### Conclusions

Could we say that our model is state of the art and the results are acceptable for the task? Definately, we can say so. Nowadays, Bi-LSTM is one of the state of the art approaches for solving NER problem and it outperforms other classical methods. Despite the fact that we used small training corpora (in comparison with usual sizes of corpora in Deep Learning), our results are quite good. In addition, in this task there are many possible named entities and for some of them we have only several dozens of trainig examples, which is definately small. However, the implemented model outperforms classical CRFs for this task. Even better results could be obtained by some combinations of several types of methods, e.g. see [this](https://arxiv.org/abs/1603.01354) paper if you are interested.